# Final Noise Analyses and Factorized Likelihood Run

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import numpy as np
import glob, os, json, string, pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import logging, inspect, copy
logging.basicConfig(level=logging.WARNING)

In [3]:
import enterprise
from enterprise.pulsar import Pulsar

import enterprise_extensions
from enterprise_extensions import models, model_utils, blocks
from enterprise_extensions.models import model_singlepsr_noise
from enterprise_extensions.chromatic import solar_wind, chromatic
from enterprise_extensions.hypermodel import HyperModel
from enterprise_extensions.sampler import setup_sampler

You'll need sksparse for get_coefficients() with common signals!


# Free Spectral Red-noise model selection on 12.5yr Dataset

## Load Pickle File 

In [4]:
psrname = 'J1911+1347'
filepath = './no_dmx_pickles/'
filepath += '{0}_ng12p5yr_v3_nodmx_ePSR.pkl'.format(psrname)
with open(filepath,'rb') as fin:
    psr=pickle.load(fin)

# Testing models with GP DM variations

__Very Import:__ What follows is an __example__ of noise model selection. For *most* pulsars the choice of noise models used in any given model selection analysis will be different than the ones chosen here. Those working on pulsars highlighted in the 11-year noise model analysis should include those models in their analyses and also use the best combination of models from that work in any final model selection that is done.

## Setup GP model selection

Load the pulsar's favored model

In [8]:
model_kwargs_path = '/Users/hazboun/nanograv_detection/ent_ext_testing/J1911+1347/nondiag_dmgp/model_kwargs.json'
with open(model_kwargs_path, 'r') as fin:
    model_kwargs = json.load(fin)
  

In [9]:
#Here I'm assumed that the model we want is the third one.
fs_kwargs = copy.deepcopy(model_kwargs['3'])
fs_kwargs.update({'psd':'spectrum'})

In [10]:
pta = model_singlepsr_noise(psr, **fs_kwargs)

### NOTE: Should use an empirical distribution made from a prior run of this model!!!

In [ ]:
emp_dist_path = './wn_emp_dists/J1911+1347_ng12p5yr_v3_std_plaw_emp_dist.pkl'

In [19]:
sampler = setup_sampler(pta, outdir='/Users/hazboun/nanograv_detection/ent_ext_testing/J1911+1347/free_spec_run/',
                        empirical_distr=emp_dist_path)

Adding empirical proposals...

Adding red noise prior draws...

Adding DM GP noise prior draws...



In [20]:
tspan = model_utils.get_tspan([psr])
achrom_freqs = np.linspace(1/tspan,30/tspan,30)
np.savetxt(sampler.outDir + 'achrom_rn_freqs.txt', achrom_freqs, fmt='%.18e')

In [21]:
with open(sampler.outDir+'/model_kwargs.json' , 'w') as fout:
    json.dump(fs_kwargs, fout, sort_keys=True,
              indent=4, separators=(',', ': '))

In [11]:
N = 500_000
x0 = np.hstack(p.sample() for p in pta_crn.params)
Sampler.sample(x0, , SCAMweight=30, AMweight=15,
               DEweight=30, burn=10000)

## Factorized Likelihood Run 

Here we substitute in the kwargs needed for a factorized likelihood analysis. Notice that the time span used here is the time span of the full data set. This ensures that the frequencies used in the red noise model and the "GWB" model are the same. The number of components is set to 5 to replicate the factorized likelihood runs from the 12.5 year analysis. 

In [14]:
fLike_kwargs = copy.deepcopy(model_kwargs['3'])
Tspan = 407576851.48121357
print(Tspan/(365.25*24*3600),' yrs')
fLike_kwargs.update({'factorized_like':True,
                     'Tspan':Tspan,
                     'fact_like_gamma':13./3,
                     'gw_components':5})

12.915331060702131  yrs


In [15]:
pta_fL = model_singlepsr_noise(psr, **fLike_kwargs)

In [22]:
sampler = setup_sampler(pta, 
                        outdir='/Users/hazboun/nanograv_detection/ent_ext_testing/J1911+1347/factorized_likelihood/',
                        empirical_distr=emp_dist_path)

Adding empirical proposals...

Adding red noise prior draws...

Adding DM GP noise prior draws...



In [23]:
achrom_freqs_fL = np.linspace(1/Tspan,10/Tspan,10)
np.savetxt(sampler.outDir + 'achrom_rn_freqs.txt', achrom_freqs_fL, fmt='%.18e')

In [24]:
with open(sampler.outDir+'/model_kwargs.json' , 'w') as fout:
    json.dump(fLike_kwargs, fout, sort_keys=True,
              indent=4, separators=(',', ': '))

In [ ]:
N = 1_000_000
x0 = np.hstack(p.sample() for p in pta_crn.params)
Sampler.sample(x0, , SCAMweight=30, AMweight=15,
               DEweight=30, burn=10000)